In [5]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf

### Load Data

In [6]:
df_train = pd.read_csv('datafiles/train.csv')
df_train.head()

,filename,text,label
0,D36051.pdf.out.html.txt,janet ley approval sow mcw dba dmi mobility s...,3
1,D07271.pdf.out.html.txt,agreement this schedule a this schedule is att...,1
2,D28723.pdf.out.html.txt,wolters kluwer contingent staffing request fo...,3
3,D42247.pdf.out.html.txt,agreement received this agreement is entered i...,2
4,D19377.pdf.out.html.txt,addendum to hosting and services agreement thi...,0


In [7]:
df_train.shape

(15472, 3)

In [8]:
df_train_filtered = df_train.text.tolist()

In [9]:
y_train = df_train.label.tolist()

In [10]:
df_test = pd.read_csv('datafiles/test.csv')
df_test.head()

,text,label
0,addendum this software maintenance support and...,1
1,cc h statement of work schedule a financial s...,3
2,wolters kluwer certification the undersigned ...,3
3,wolters kluwer westpoint statement of work fo...,3
4,wolters kluwer contingent staffing request fo...,3


In [11]:
y_test = df_test.label.tolist()

In [12]:
df_train_latent = pd.read_csv('../xgboost/datafiles/latent_features_train.csv')
df_train_latent.head()

,DocID,addendum date,addendum effective date,addendum entered,addendum made,addendum number,addendum_keywords_count,addendum_regex_lookup,affiliated entity,agreement hereby amended,...,sow shall,sow term,sow_keywords_count,sow_regex_lookup,statement of work effective,subject matter hereof,subsequent addendum,term of addendum,term of amendment,work sow
0,D36051.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,D07271.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D28723.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,D42247.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D19377.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_train_latent = df_train_latent.drop(['DocID'], axis=1)
df_train_latent.shape

(15472, 64)

In [14]:
df_test_latent = pd.read_csv('../xgboost/datafiles/latent_features_test.csv')
df_test_latent.head()

,DocID,addendum date,addendum effective date,addendum entered,addendum made,addendum number,addendum_keywords_count,addendum_regex_lookup,affiliated entity,agreement hereby amended,...,sow shall,sow term,sow_keywords_count,sow_regex_lookup,statement of work effective,subject matter hereof,subsequent addendum,term of addendum,term of amendment,work sow
0,D00152.pdf.out.html.txt,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,D16833.pdf.out.html.txt,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,D08368.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,D38435.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D00918.pdf.out.html.txt,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_test_latent = df_test_latent.drop(['DocID'], axis=1)
df_test_latent.shape

(1397, 64)

In [30]:
df_labeled = pd.read_csv('../datafiles/labeled_data_relabeled.csv')
df_labeled.head()

,filename,text,label
0,D00152.pdf.out.html.txt,addendum no to master services agreement this ...,Addendum
1,D16833.pdf.out.html.txt,addendum no to master services agreement this ...,Addendum
2,D08368.pdf.out.html.txt,agreement i to add effective as of january the...,Addendum
3,D38435.pdf.out.html.txt,amendment number one to work order this amendm...,Addendum
4,D00918.pdf.out.html.txt,agreement and or enrollment identified above t...,Addendum


In [31]:
df_labeled.shape

(1397, 3)

### Training

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 512 # cut texts after this number of words
latent_features_size = df_train_latent.shape[1] 

# prepare tokenizer
t = Tokenizer(num_words=max_features)
t.fit_on_texts(df_train_filtered)
#t.fit_on_texts(df_valid)
post_seq_train = t.texts_to_sequences(df_train_filtered)
#post_seq_valid = t.texts_to_sequences(df_valid)
X_train = pad_sequences(post_seq_train, maxlen=max_features, padding='post')
#X_valid = pad_sequences(post_seq_valid, maxlen=max_features, padding='post')

Using TensorFlow backend.


In [15]:
#print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
print(X_train.shape, len(y_train))

(15472, 512) 15472


In [16]:
word_index = t.word_index
len(word_index)

401227

In [ ]:
#Import Embeddings and create embedding dict
embeddings_index = {}
f = open('glove.6B/glove.6B.300d.txt', encoding="utf8")
#f = open('../w2v_embedding/cms_word2vec_embedding_300.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [1]:
from keras.layers import Embedding
from keras.initializers import Constant

#Create embedding layer
EMBEDDING_DIM = 300
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_features,
                            trainable=False)

Using TensorFlow backend.
/home/user/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.7) or chardet (2.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


KeyboardInterrupt: 

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=512)
X_train = vectorizer.fit_transform(df_train_filtered)

#X_dev = vectorizer.transform(df_dev.text.tolist())
#X_valid = vectorizer.transform(df_valid)
X_test = vectorizer.transform(df_labeled.text.tolist())

In [22]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Bidirectional, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate

max_features = 401227
batch_size = 32
latent_features_size = df_train_latent.shape[1] 

"""model = Sequential()
model.add(Embedding(max_features, 128, input_length=100))
#model.add(Embedding(len(word_index), 128, input_length=max_features, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(256)))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.add(concatenate([bi_lstm_model, inputs_latent], axis=1))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
"""

inputs = Input(shape=(512,))
inputs_latent = Input(shape=(latent_features_size,))
embed = Embedding(401227, 128)(inputs)
bi_lstm_model = Bidirectional(LSTM(256))(embed)
x = concatenate([bi_lstm_model, inputs_latent], axis=1)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(5, activation='softmax')(x)
model = Model(inputs = [inputs, inputs_latent], outputs = x)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 512, 128)     51357056    input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 512)          788480      embedding_2[0][0]                
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 64)           0                                            
____________________________________________________________________________________________

In [32]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
lr = 3e-4
epochs = 10
#decay=lr/epochs
adam = Adam(lr=lr)

model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

In [33]:
model_chkpoint = ModelCheckpoint(filepath='best_model.hdf5', save_best_only=True, save_weights_only=True)

In [34]:
reduce_lr = ReduceLROnPlateau(monitor='val_acc', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-7)

In [35]:
import pandas as pd
T_train = pd.get_dummies(y_train)
#T_valid = pd.get_dummies(y_valid)

In [36]:
model.fit([X_train, df_train_latent], T_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True, 
          validation_split=0.03,
          callbacks=[reduce_lr, model_chkpoint])

Train on 15007 samples, validate on 465 samples
Epoch 1/10
15007/15007 [==============================] - 442s 29ms/step - loss: 0.3829 - acc: 0.8873 - val_loss: 0.1556 - val_acc: 0.9828
Epoch 2/10
15007/15007 [==============================] - 440s 29ms/step - loss: 0.1025 - acc: 0.9719 - val_loss: 0.0419 - val_acc: 0.9957
Epoch 3/10
15007/15007 [==============================] - 440s 29ms/step - loss: 0.0646 - acc: 0.9825 - val_loss: 0.0188 - val_acc: 0.9935
Epoch 4/10
15007/15007 [==============================] - 441s 29ms/step - loss: 0.0473 - acc: 0.9867 - val_loss: 0.0136 - val_acc: 0.9978
Epoch 5/10
15007/15007 [==============================] - 445s 30ms/step - loss: 0.0436 - acc: 0.9874 - val_loss: 0.0129 - val_acc: 0.9957
Epoch 6/10
15007/15007 [==============================] - 458s 31ms/step - loss: 0.0388 - acc: 0.9895 - val_loss: 0.0127 - val_acc: 0.9957
Epoch 7/10
15007/15007 [==============================] - 480s 32ms/step - loss: 0.0309 - acc: 0.9927 - val_loss: 0.04

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Bidirectional
from utils import get_keras_early_stopping
from keras.optimizers import Adam

def create_model(lr=0.001):
    # create model
    model = Model()
    inputs = Input(shape=(max_features,), dtype='int32')
    embed = embedding_layer(inputs)
    x = Bidirectional(LSTM(64))(embed)
    x = Dropout(0.5)(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(inputs = inputs, outputs = x)
    
    adam = Adam(lr=lr)
    
    # Compile model
    model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, batch_size=32, epochs=15, verbose=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

# define the grid search parameters
#batch_size = [10, 20, 40, 60]
#epochs = [10, 25, 50]
lr = [0.001, 0.01, 0.1]
param_grid = dict(lr=lr)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=5,  verbose=10)
grid_result = grid.fit(X_train, y_train)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Predictions

In [33]:
model.load_weights('models/lstm-10epochs-83-100-99.37.hdf5')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 512, 128)     51357056    input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 512)          788480      embedding_2[0][0]                
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 64)           0                                            
____________________________________________________________________________________________

In [34]:
"""text_pred = df_labeled.text.tolist()
t.fit_on_texts(text_pred)
post_seq_test = t.texts_to_sequences(text_pred)
x_test = pad_sequences(post_seq_test, maxlen=max_features, padding='post')"""
probs_test = model.predict([X_test, df_test_latent])

In [35]:
labels_test = df_labeled.label.map({'Addendum': 0, 'MSA': 1, 'SOW': 4, 'NDA': 2, 'Others': 3})
len(labels_test)

1397

In [36]:
from snorkel.analysis import metric_score
test_acc = metric_score(golds=labels_test, preds=probs_test.argmax(axis=1), metric="accuracy")
print(f"Test Accuracy: {test_acc * 100:.1f}%")

Test Accuracy: 88.0%


In [37]:
df_labeled['preds'] = pd.Series(probs_test.argmax(axis=1)).map({0: 'Addendum', 1: 'MSA', 4: 'SOW', 2: 'NDA', 3: 'Others'})

In [38]:
df_labeled.to_csv('../predictions/lstm_predictions_after_relabeling.csv', index=False)

### Save Model

In [ ]:
from keras.callbacks.callbacks import ModelCheckpoint
filepath = 'models/lstm.hdf5'
checkpointer = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

### Save Model Probabilities and Labels

In [ ]:
train_pred = df_unlabeled.text.tolist()
t.fit_on_texts(train_pred)
post_seq_test = t.texts_to_sequences(train_pred)
x_train = pad_sequences(post_seq_test, maxlen=max_features, padding='post')
probs_train = model.predict(x_train)

In [ ]:
len(probs_train)

In [ ]:
labels_train = label_model.predict(L_unlabelled)
labels_train

In [ ]:
print(len(labels_train), len(labels_test))
print(len(probs_train), len(probs_test))

In [ ]:
probs_latent_train = list(zip(probs_train.tolist(), labels_train.tolist()))
probs_latent_test = list(zip(probs_test.tolist(), labels_test.tolist()))
probs_latent_test

In [ ]:
df_probs_train = pd.DataFrame(probs_latent_train, columns=['probabilities', 'label'])
df_probs_test = pd.DataFrame(probs_latent_test, columns=['probabilities', 'label'])

In [ ]:
df_probs_test.head()

In [ ]:
df_probs_train.to_csv('probabilities_train.csv', index = None)
df_probs_test.to_csv('probabilities_test.csv', index = None)